# Colab: Prueba rápida del proyecto ML_RL

Notebook de Colab que clona el repositorio, instala dependencias (Colab-friendly), ejecuta un smoke test y corre un ejemplo corto de `test.py` o `train.py`.

Repositorio por defecto: `https://github.com/Escortiz/ML_RL.git` — cambia la URL si quieres usar un fork o una copia en Drive.


In [ ]:
# 1) Configurar variables del repositorio
REPO_URL = "https://github.com/Escortiz/ML_RL.git"
BRANCH = "main"  # dejar vacío para usar la rama por defecto
TARGET_DIR = "repo"

print(f"REPO_URL = {REPO_URL}")
print(f"BRANCH = {BRANCH}")
print(f"TARGET_DIR = {TARGET_DIR}")

# Si prefieres clonar desde Drive, define TARGET_DIR con la ruta de Drive, por ejemplo:
# TARGET_DIR = '/content/drive/MyDrive/ML_RL'

## 2) Montar Google Drive (opcional)

Si quieres guardar modelos o videos persistentes en tu Drive, monta Google Drive y crea la carpeta destino. Ejecuta esta celda sólo si necesitas persistencia entre sesiones.

```python
from google.colab import drive
# drive.mount('/content/drive')
# !mkdir -p /content/drive/MyDrive/ML_RL_outputs
```


In [ ]:
# 3) Clonar repositorio desde GitHub y cambiar directorio
import os
import sys

print('Clonando', REPO_URL)
# si el directorio ya existe intentamos hacer pull
if os.path.exists(TARGET_DIR):
    print(f"{TARGET_DIR} existe — actualizando (git pull)")
    %cd $TARGET_DIR
    !git pull || echo 'git pull falló'
else:
    !git clone {REPO_URL} {TARGET_DIR}
    %cd $TARGET_DIR

# opcional: cambiar rama
if BRANCH:
    !git checkout {BRANCH} || echo 'git checkout falló o rama inexistente'

## 4) Actualizar sistema e instalar dependencias de renderizado

Ejecuta esto si vas a usar render/headless o necesitas compilar paquetes que requieren librerías del sistema. Puedes omitir esta celda si no es necesario.

In [ ]:
# 4) System deps (apt) - actualiza e instala librerías para render/headless
!apt-get update -y
!apt-get install -y libosmesa6-dev libgl1-mesa-glx libglfw3 swig ffmpeg || echo 'apt-get install falló - intenta re-ejecutar'

## 5) Instalar/actualizar herramientas de Python y PyTorch

Actualiza pip y herramientas de empaquetado y luego instala PyTorch (CPU por defecto). Si quieres GPU, cambia el comando según las instrucciones en https://pytorch.org/.

In [ ]:
# upgrade packaging tools and install cython + torch (CPU)
!pip install -U pip setuptools wheel
!pip install -U cython
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Si quieres GPU, reemplaza la línea anterior por el comando recomendado en pytorch.org para la versión de CUDA disponible.

## 6) Instalar dependencias del proyecto (requirements_colab.txt)

Instala las dependencias adaptadas a Colab. Si falla la instalación por `gym`, el fallback comentado intenta una versión compatible.

In [ ]:
# 6) Instalar dependencias del proyecto
!pip install -r requirements_colab.txt || echo 'pip install -r requirements_colab.txt falló'

# Fallback: si falla por gym, intenta una versión alternativa (descomenta si hace falta)
# !pip install gym==0.26.2

## 7) Configuración de MuJoCo (opcional)

Si usas MuJoCo moderno puedes instalar la librería `mujoco` y configurar el render headless. Si tu proyecto usa `mujoco-py`, sigue la documentación de `mujoco-py` y añade la licencia si es necesario.

In [ ]:
# 7) MuJoCo moderno (opcional)
# !pip install mujoco
# %env MUJOCO_GL=egl

print('Si usas mujoco-py puede requerir pasos adicionales y una licencia; consulta la documentación.')

## 8) Smoke test rápido del agente

Esta celda prueba que `agent.Policy` carga y hace un forward rápido con tensores de ejemplo. Si falla, revisa la salida del stack trace para ver import errors o fallos de dependencias.

In [ ]:
# Smoke test
import torch
from agent import Policy

p = Policy(11, 3)
s = torch.zeros(1, 11)
d, v = p(s)
print('mean', d.mean.shape, 'std', d.stddev.shape, 'value', v.shape)

## 9) Ejecutar test/entrenamiento corto

Ejecuta un test corto para verificar el loop de evaluación. Puedes ajustar variables arriba para cambiar número de episodios o ruta del modelo.

In [ ]:
# CELDA: Ejecutar test parametrizado
EPISODES = 2
MODEL_PATH = './model.mdl'  # cambia a la ruta de tu modelo si es necesario

print(f"Ejecutando test con EPISODES={EPISODES}, MODEL_PATH={MODEL_PATH}")
!python test.py --episodes {EPISODES} --model {MODEL_PATH}

# Si quieres ejecutar en background (no recomendado en Colab interactivo), usa:
# !nohup python test.py --episodes {EPISODES} --model {MODEL_PATH} > test_log.txt 2>&1 &


In [ ]:
# CELDA: Ejecutar train parametrizado
N_EPISODES = 10
DEVICE = 'cpu'  # o 'cuda' si instalaste torch con soporte CUDA

print(f"Ejecutando train con N_EPISODES={N_EPISODES}, DEVICE={DEVICE}")
# Ejecuta un entrenamiento corto
!python train.py --n-episodes {N_EPISODES} --device {DEVICE}

# Para ejecutar en background en Colab usar nohup (solo si necesitas desatender el proceso):
# !nohup python train.py --n-episodes {N_EPISODES} --device {DEVICE} > train_log.txt 2>&1 &
